In [ ]:
# Copyright 2025 Forusone

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Engine on Vertex AI


### Agent Engine in Vertex AI

### Install Vertex AI SDK for Python

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index]

In [ ]:
# Note:  Need to restart the kernel
%pip install --upgrade --quiet langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00


#### Note: <font color="red">Need to restart the kernel after installing</font>

In [ ]:
exit()

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

## Initial set up

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://agent-0417/...
ServiceException: 409 A Cloud Storage bucket named 'agent-0417' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [ ]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai
from vertexai import agent_engines
# import langchain_google_vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Tool

In [ ]:
# @title Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [ ]:
# Tool test
get_exchange_rate(currency_from="USD", currency_to="KRW")

{'amount': 1.0, 'base': 'USD', 'date': '2025-04-15', 'rates': {'KRW': 1426.85}}

## Local Agent

In [ ]:
# @title Agent Define - Local unit test
# from vertexai.preview.reasoning_engines import LangchainAgent
from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.LangchainAgent

local_agent = agent_engines.LangchainAgent(
    model = MODEL_NAME,
    model_kwargs={
      "temperature": 0.28,
      "max_output_tokens": 1000,
      "top_p": 0.95,
      "top_k": None,
    },
    tools = [get_exchange_rate],
    # agent_executor_kwargs={"return_intermediate_steps": False},
)

In [ ]:
# @title Test your agent locally

# Agent local unit test before deploying.
local_agent.query(input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?")

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-04-15 is 1426.85.\n'}

## Remote Agent

In [ ]:
# @title Deploy your agent on Vertex AI

from vertexai import agent_engines

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines.ReasoningEngine#vertexai_preview_reasoning_engines_ReasoningEngine_create

remote_agent = agent_engines.create(
    local_agent,
    display_name="currency agent",
    gcs_dir_name = "ai-agent-1",
    description="This is a simple reasoning engine.",
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain,ag2]",
        "langchain-google-vertexai",
        "cloudpickle==3.0",
    ],
    extra_packages = []
)

INFO:vertexai.agent_engines:Identified the following requirements: {'cloudpickle': '3.1.1', 'google-cloud-aiplatform': '1.88.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain,ag2]', 'langchain-google-vertexai', 'cloudpickle==3.0']
INFO:vertexai.agent_engines:Using bucket agent-0417
INFO:vertexai.agent_engines:Wrote to gs://agent-0417/ai-agent-1/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-1/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent-0417/ai-agent-1/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/721521243942/locations/us-central1/reasoningEngines/1965407820975177728/operations/3422586836055228416
INFO:vertexai.agent_engines:View progress and logs at https://console.cloud.google.com/logs/query?project=ai-

In [ ]:
# @title Query from remote engine.
remote_agent.query(
    input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?"
)

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-04-15 is 1426.85.\n'}

## Agent Management

In [ ]:
# @title Helper function to manage reasoning engine.

# Properties of ReasoningEngine class.
# https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview

#----------------------------------------
def get_agent_engine(display_name:str):
  try:
    for agent in agent_engines.list():
      if agent.display_name == display_name:
        return agent_engines.get(agent.name)

      else:
        print("No such reasoning engine or Invalid display name.")

  except Exception as e:
    print(e)

#----------------------------------------
def show_agents():
  """
  List reasoning engines.
  """

  try:
    if not agent_engines.list():
      print("No reasoning engines")

    for idx, agent in enumerate(agent_engines.list()):
        print(f"Agent {idx}: \n\tDisplay Name [{agent.display_name}] \n\tName [{agent.name}] \n\tCreation Time [{agent.create_time}] \n\tResource Name [{agent.resource_name}]\n")

  except Exception as e:
    print(e)

#----------------------------------------

def delete_agent(name):
  """
  Delete a reasoning engines.
  @param name: The name of the reasoning engine.
  @type name: str
  """

  try:
    re = agent_engines.get(name)
    re.delete()
    print(f"Deleted {name}")
  except Exception as e:
    print(e)


In [ ]:
show_agents()

Agent 0: 
	Display Name [currency agent] 
	Name [1965407820975177728] 
	Creation Time [2025-04-16 04:31:34.165206+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/1965407820975177728]

Agent 1: 
	Display Name [currency agent with adk] 
	Name [4968182872524455936] 
	Creation Time [2025-04-16 03:28:46.545118+00:00] 
	Resource Name [projects/721521243942/locations/us-central1/reasoningEngines/4968182872524455936]



In [ ]:
get_agent_engine(display_name="currency agent")


resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/1965407820975177728

In [ ]:
delete_agent(name="712483534798848")

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/721521243942/locations/us-central1/operations/2269665331448381440
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/ai-hangsik/locations/us-central1/reasoningEngines/712483534798848


Deleted 712483534798848


In [ ]:
# @title Get instances with display name.
new_remote_agent = get_agent_engine('currency agent')

new_remote_agent.query(
    input="What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?"
)

{'input': "What's the exchange rate from US dollars(USD) to Korean currency(KRW) at 2025-04-15 ?",
 'output': 'OK. The exchange rate from USD to KRW on 2025-04-15 is 1426.85.\n'}

## End of notebook